# Project 3: Understanding User Behavior
**Project Team: Jude Wentian Zhu, Rohit Barkshi, Rathin Bector**

## Description of Project Files


<span style="color:red">**Need to fill in**</span>

## Project Goal

<span style="color:red">**Need to fill in**</span>

## Summary of Data Pipeline

<span style="color:red">**Need to fill in**</span>

## Data Pipeline Steps

### Step 1: Spin up Docker-Compose and Link Pypark to Jupyter Notebook

1. Spin up the cluster

In [1]:
!docker-compose up -d

Creating network "project_3_default" with the default driver
Creating project_3_presto_1 ... 
Creating project_3_redis_1  ... 
Creating project_3_cloudera_1 ... 
Creating project_3_zookeeper_1 ... 
Creating project_3_mids_1      ... 
ting project_3_cloudera_1  ... doneCreating project_3_spark_1     ... 
Creating project_3_kafka_1     ... mdone
ting project_3_kafka_1     ... done

2. Exec a bash shell in the spark container on teminal.
```console
docker-compose exec spark bash
```


3. Create a symbolic link from the spark directory to /w205 :
```console
ln -s /w205 w205
```


4. Exit the container
```console
exit
```


5. Check out Hadoop

In [2]:
!docker-compose exec cloudera hadoop fs -ls /tmp/

Found 3 items
drwxrwxrwt   - mapred mapred              0 2016-04-06 02:26 /tmp/hadoop-yarn
drwx-wx-wx   - hive   supergroup          0 2021-08-05 04:36 /tmp/hive
drwxrwxrwt   - mapred hadoop              0 2016-04-06 02:28 /tmp/logs


### Step 2: Launch Kafka and Flask

1. Create a kafka topic called events

In [3]:
!docker-compose exec kafka kafka-topics --create --topic events --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181

Created topic events.


2. Install dependencies for flask app on mids container

In [4]:
!docker-compose exec mids pip install redis
!docker-compose exec mids pip install numpy==1.14.6

The directory '/w205/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/w205/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
    100% |################################| 81kB 6.1MB/s eta 0:00:01
You are using pip version 8.1.1, however version 21.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
The directory '/w205/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/w205/.cache/pip' or its parent directory is not 

3. Run flask app on teminal
```console
docker-compose exec mids env FLASK_APP=/w205/project_3/game_api.py flask run --host 0.0.0.0
```

In [ ]:
!bash data_generator.sh

5. read events from Kafka

In [1]:
!docker-compose exec mids kafkacat -C -b kafka:29092 -t events -o beginning -e

/bin/sh: 1: docker-compose: not found


### Step 3: Data Streaming


In [11]:
# Import Packages
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, from_json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType


In [52]:
#defin schema
def player_event_schema():
    """
    root
    |-- Accept: string (nullable = true)
    |-- Host: string (nullable = true)
    |-- User-Agent: string (nullable = true)
    |-- event_type: string (nullable = true)
    |-- username: string (nullable = true)
    |-- timestamp: string (nullable = true)
    """
    return StructType([
        StructField("Accept", StringType(), True),
        StructField("Host", StringType(), True),
        StructField("User-Agent", StringType(), True),
        StructField("event_type", StringType(), True),
        StructField("username", StringType(), True),
        StructField("timestamp", StringType(), True),
    ])

def purchase_weapon_event_schema():
    """
    root
    |-- Accept: string (nullable = true)
    |-- Host: string (nullable = true)
    |-- User-Agent: string (nullable = true)
    |-- event_type: string (nullable = true)
    |-- username: string (nullable = true)
    |-- weapon: string (nullable = true)
    |-- wallet_before:integer  (nullable = true)
    |-- wallet_after:integer  (nullable = true)
    |-- timestamp: string (nullable = true)
    """
    return StructType([
        StructField("Accept", StringType(), True),
        StructField("Host", StringType(), True),
        StructField("User-Agent", StringType(), True),
        StructField("event_type", StringType(), True),
        StructField("username", StringType(), True),
        StructField("weapon", StringType(), True),
        StructField("wallet_before", IntegerType(), True),
        StructField("wallet_after", IntegerType(), True),
        StructField("timestamp", StringType(), True),
    ])

def purchase_shield_event_schema():
    """
    root
    |-- Accept: string (nullable = true)
    |-- Host: string (nullable = true)
    |-- User-Agent: string (nullable = true)
    |-- event_type: string (nullable = true)
    |-- username: string (nullable = true)
    |-- wallet_before:integer  (nullable = true)
    |-- wallet_after:integer  (nullable = true)
    |-- timestamp: string (nullable = true)
    """
    return StructType([
        StructField("Accept", StringType(), True),
        StructField("Host", StringType(), True),
        StructField("User-Agent", StringType(), True),
        StructField("event_type", StringType(), True),
        StructField("username", StringType(), True),
        StructField("wallet_before", IntegerType(), True),
        StructField("wallet_after", IntegerType(), True),
        StructField("timestamp", StringType(), True),
    ])

def dig_for_gold_event_schema():
    """
    root
    |-- Accept: string (nullable = true)
    |-- Host: string (nullable = true)
    |-- User-Agent: string (nullable = true)
    |-- event_type: string (nullable = true)
    |-- username: string (nullable = true)
    |-- gold_found: integer (nullable = true)
    |-- wallet_before:integer  (nullable = true)
    |-- wallet_after:integer  (nullable = true)
    |-- timestamp: string (nullable = true)
    """
    return StructType([
        StructField("Accept", StringType(), True),
        StructField("Host", StringType(), True),
        StructField("User-Agent", StringType(), True),
        StructField("event_type", StringType(), True),
        StructField("username", StringType(), True),
        StructField("gold_found", IntegerType(), True),
        StructField("wallet_before", IntegerType(), True),
        StructField("wallet_after", IntegerType(), True),
        StructField("timestamp", StringType(), True),
    ])

def successful_attack_event_schema():
    """
    root
    |-- Accept: string (nullable = true)
    |-- Host: string (nullable = true)
    |-- User-Agent: string (nullable = true)
    |-- event_type: string (nullable = true)
    |-- attacker: string (nullable = true)
    |-- defender: string (nullable = true)
    |-- weapon_used: string (nullable = true)
    |-- defender_has_shield: boolean (nullable = true)
    |-- defender_health_before:integer  (nullable = true)
    |-- defender_health_after:integer  (nullable = true)
    |-- defender_killed: boolean (nullable = true)
    |-- timestamp: string (nullable = true)
    """
    return StructType([
        StructField("Accept", StringType(), True),
        StructField("Host", StringType(), True),
        StructField("User-Agent", StringType(), True),
        StructField("event_type", StringType(), True),
        StructField("attacker", StringType(), True),
        StructField("defender", StringType(), True),
        StructField("weapon_used", StringType(), True),
        StructField("defender_has_shield", BooleanType(), True),
        StructField("defender_health_before", IntegerType(), True),
        StructField("defender_health_after", IntegerType(), True),
        StructField("defender_killed", BooleanType(), True),
        StructField("timestamp", StringType(), True),
    ])

def failed_attack_event_schema():
    """
    root
    |-- Accept: string (nullable = true)
    |-- Host: string (nullable = true)
    |-- User-Agent: string (nullable = true)
    |-- event_type: string (nullable = true)
    |-- attacker: string (nullable = true)
    |-- defender: string (nullable = true)
    |-- weapon_used: string (nullable = true)
    |-- timestamp: string (nullable = true)
    """
    return StructType([
        StructField("Accept", StringType(), True),
        StructField("Host", StringType(), True),
        StructField("User-Agent", StringType(), True),
        StructField("event_type", StringType(), True),
        StructField("attacker", StringType(), True),
        StructField("defender", StringType(), True),
        StructField("weapon_used", StringType(), True),
        StructField("timestamp", StringType(), True),
    ])

In [137]:
#define functions
@udf('boolean')
def is_player(event_as_json):
    event = json.loads(event_as_json)
    if event['event_type'] == 'initialize_player':
        return True
    return False


@udf('boolean')
def is_purchase_weapon(event_as_json):
    event = json.loads(event_as_json)
    # m = re.match('purchase',event['event_type'])
    if event['event_type'] == 'purchase_weapon':
        return True
    return False

@udf('boolean')
def is_purchase_shield(event_as_json):
    event = json.loads(event_as_json)
    if event['event_type'] == 'purchase_shield':
        return True
    return False


@udf('boolean')
def is_dig_for_gold(event_as_json):
    event = json.loads(event_as_json)
    if event['event_type'] == 'dig_for_gold':
        return True
    return False


@udf('boolean')
def is_purchase_shield(event_as_json):
    event = json.loads(event_as_json)
    if event['event_type'] == 'purchase_shield':
        return True
    return False

@udf('boolean')
def is_successful_attack(event_as_json):
    event = json.loads(event_as_json)
    if event['event_type'] == 'successful_attack':
        return True
    return False

@udf('boolean')
def is_failed_attack(event_as_json):
    event = json.loads(event_as_json)
    if event['event_type'] == 'failed_attack':
        return True
    return False


Start reading events

### Batch

In [145]:
spark = SparkSession \
    .builder \
    .appName("ExtractEventsJob") \
    .getOrCreate()

raw_events = spark \
    .read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "events") \
    .option("startingOffsets", "earliest") \
    .option("endingOffsets", "latest") \
    .load()

In [146]:
initialize_player = raw_events \
    .filter(is_player(raw_events.value.cast('string'))) \
    .select(raw_events.value.cast('string').alias('raw_event'),
            raw_events.timestamp.cast('string'),
            from_json(raw_events.value.cast('string'),
                      player_event_schema()).alias('json')) \
    .select('raw_event', 'timestamp', 'json.*')

dig_for_gold = raw_events \
    .filter(is_dig_for_gold(raw_events.value.cast('string'))) \
    .select(raw_events.value.cast('string').alias('raw_event'),
        raw_events.timestamp.cast('string'),
        from_json(raw_events.value.cast('string'),
                  dig_for_gold_event_schema()).alias('json')) \
.select('raw_event', 'timestamp', 'json.*')


In [103]:
import pandas as pd
pd.set_option('display.max_colwidth', 1000)

In [147]:
initialize_player.show()

+--------------------+--------------------+------+-----------------+---------------+-----------------+--------+---------+
|           raw_event|           timestamp|Accept|             Host|     User-Agent|       event_type|username|timestamp|
+--------------------+--------------------+------+-----------------+---------------+-----------------+--------+---------+
|{"username": "use...|2021-08-05 20:07:...|   */*|user1.comcast.com|ApacheBench/2.3|initialize_player|  user50|     null|
|{"username": "use...|2021-08-05 20:07:...|   */*|user1.comcast.com|ApacheBench/2.3|initialize_player|  user29|     null|
|{"username": "use...|2021-08-05 20:53:...|   */*|user1.comcast.com|ApacheBench/2.3|initialize_player|  user36|     null|
|{"username": "use...|2021-08-05 20:53:...|   */*|user1.comcast.com|ApacheBench/2.3|initialize_player|  user41|     null|
|{"username": "use...|2021-08-05 20:54:...|   */*|user1.comcast.com|ApacheBench/2.3|initialize_player|  user20|     null|
|{"username": "use...|20

In [148]:
dig_for_gold.show()

+--------------------+--------------------+------+-----------------+---------------+------------+--------+----------+-------------+------------+---------+
|           raw_event|           timestamp|Accept|             Host|     User-Agent|  event_type|username|gold_found|wallet_before|wallet_after|timestamp|
+--------------------+--------------------+------+-----------------+---------------+------------+--------+----------+-------------+------------+---------+
|{"username": "use...|2021-08-05 18:26:...|   */*|user1.comcast.com|ApacheBench/2.3|dig_for_gold|   user3|        19|          100|         114|     null|
|{"username": "use...|2021-08-05 18:26:...|   */*|user1.comcast.com|ApacheBench/2.3|dig_for_gold|   user3|         3|          114|         112|     null|
|{"username": "use...|2021-08-05 18:26:...|   */*|user1.comcast.com|ApacheBench/2.3|dig_for_gold|   user3|        43|          112|         150|     null|
|{"username": "use...|2021-08-05 18:26:...|   */*|user1.comcast.com|Ap

In [157]:
spark = SparkSession \
    .builder \
    .appName("ExtractEventsJob") \
    .getOrCreate()
    
raw_events = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "events") \
    .load()

In [158]:
#extract initial_player events
extracted_initialize_player = raw_events \
    .filter(is_player(raw_events.value.cast('string'))) \
    .select(raw_events.value.cast('string').alias('raw_event'),
            raw_events.timestamp.cast('string'),
            from_json(raw_events.value.cast('string'),
                      player_event_schema()).alias('json')) \
    .select('raw_event', 'timestamp', 'json.*')

sink_player = extracted_initialize_player \
    .writeStream \
    .format("parquet") \
    .option("path","/tmp/player") \
    .option("checkpointLocation", "/tmp/checkpoint_player") \
    .start()

#extract dig for gold events
extracted_dig_for_gold = raw_events \
    .filter(is_dig_for_gold(raw_events.value.cast('string'))) \
    .select(raw_events.value.cast('string').alias('raw_event'),
            raw_events.timestamp.cast('string'),
            from_json(raw_events.value.cast('string'),
                  dig_for_gold_event_schema()).alias('json')) \
    .select('raw_event', 'timestamp', 'json.*')
    
sink_dig_for_gold = extracted_dig_for_gold \
    .writeStream \
    .format("parquet") \
    .option("path","/tmp/dig_for_gold") \
    .option("checkpointLocation", "/tmp/checkpoint_dig_for_gold") \
    .start()
    
    
#extract purachase_weapon events
extracted_purchase_weapon = raw_events \
    .filter(is_purchase_weapon(raw_events.value.cast('string'))) \
    .select(raw_events.value.cast('string').alias('raw_event'),
            raw_events.timestamp.cast('string'),
            from_json(raw_events.value.cast('string'),
                      purchase_weapon_event_schema()).alias('json')) \
    .select('raw_event', 'timestamp', 'json.*')
    
sink_purchase_weapon = extracted_purchase_weapon \
    .writeStream \
    .format("parquet") \
    .option("path","/tmp/purchase_weapon") \
    .option("checkpointLocation", "/tmp/checkpoint_purchase_weapon") \
    .start()

    
#extract purachase_shield events
extracted_purchase_shield = raw_events \
    .filter(is_purchase_shield(raw_events.value.cast('string'))) \
    .select(raw_events.value.cast('string').alias('raw_event'),
            raw_events.timestamp.cast('string'),
            from_json(raw_events.value.cast('string'),
                      purchase_shield_event_schema()).alias('json')) \
    .select('raw_event', 'timestamp', 'json.*')
            
sink_purchase_shield = extracted_purchase_shield \
    .writeStream \
    .format("parquet") \
    .option("path","/tmp/purchase_shield") \
    .option("checkpointLocation", "/tmp/checkpoint_purchase_shield") \
    .start()
            
#extract successful_attack events
extracted_successful_attack = raw_events \
    .filter(is_successful_attack(raw_events.value.cast('string'))) \
    .select(raw_events.value.cast('string').alias('raw_event'),
            raw_events.timestamp.cast('string'),
            from_json(raw_events.value.cast('string'),
                      successful_attack_event_schema()).alias('json')) \
    .select('raw_event', 'timestamp', 'json.*')
    
sink_successful_attack= extracted_successful_attack \
    .writeStream \
    .format("parquet") \
    .option("path","/tmp/successful_attack") \
    .option("checkpointLocation", "/tmp/checkpoint_successful_attack") \
    .start()
            
            
#extract failed_attack events
extracted_failed_attack = raw_events \
    .filter(is_failed_attack(raw_events.value.cast('string'))) \
    .select(raw_events.value.cast('string').alias('raw_event'),
            raw_events.timestamp.cast('string'),
            from_json(raw_events.value.cast('string'),
                      failed_attack_event_schema()).alias('json')) \
    .select('raw_event', 'timestamp', 'json.*')
    
sink_failed_attack= extracted_failed_attack \
    .writeStream \
    .format("parquet") \
    .option("path","/tmp/failed_attack") \
    .option("checkpointLocation", "/tmp/checkpoint_failed_attack") \
    .start()

In [55]:
sink_player.stop()
sink_purchase_weapon.stop()

In [56]:
transformed_player_events = spark.read.parquet('/tmp/player')
transformed_player_events.registerTempTable('player')

transformed_player_events.toPandas()
transformed_player_events.head(4)

AnalysisException: "Reference 'timestamp' is ambiguous, could be: timestamp#4415, timestamp#4421.;"

### Step 3: Use Spark to Batch Analysis